In [2]:
import time
from tqdm import tqdm
import os
import sys

# Удобноее импортирование модулей из scripts
wd = os.getcwd()
wd = wd[:-wd[::-1].find("\\")]
wd_data = "data\\"
sys.path.append(wd + 'scripts')
from Paeser import yand

In [7]:
#Сбор данных

In [ ]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
def create_web_driver_stealth() -> webdriver.Chrome:
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized") 
    #options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--enable-javascript")
    options.add_argument("--lang=en-US")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    s = Service(ChromeDriverManager().install())
    stealth_driver = webdriver.Chrome(service=s, options=options)
    
    with open('navigator_config.json', 'r') as file:
        nav_data = json.load(file)
        del nav_data['plugins']
        del nav_data['mimeTypes']
    
    def get_com(i):
        return """Object.defineProperty(navigator, '%s', {get: () => "%s"});
        """ % (i[0], i[1])
    
    js_script = ''.join([get_com(i) for i in nav_data.items()])
    

    # Подмена navigator properties
    stealth_driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {get: () => false});
    Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3]});
    Object.defineProperty(navigator, 'languages', {get: () => ['ru-RU', 'ru']});
    """
    })

    stealth_driver.execute_script(js_script)
    
    
    return stealth_driver

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [15]:
url = "https://market.yandex.ru"
list_request = ["Ноутбук", "Нетбук", "Игровой ноутбук", "laptop"]

df = pd.DataFrame({'title': [], 'about': [], 'price': [],'rating': [],'reviews':[],'market': [],'link': [],'req': []})

start_time = time.perf_counter()  # про time.perf_counter https://superfastpython.com/time-time-vs-time-perf_counter/
driver = create_web_driver_stealth()

driver.get(url)
time.sleep(5)
actions = ActionChains(driver)
actions.move_by_offset(10, 10).click().perform()

print(url)
for req in tqdm(list_request):
    if url == "https://market.yandex.ru":
        df = yand(req, driver, df)
    
duration = time.perf_counter() - start_time
    
print(f"Downloaded site in {duration} seconds")

https://market.yandex.ru


100%|██████████| 4/4 [00:34<00:00,  8.61s/it]

Downloaded site in 42.998212800001056 seconds


In [26]:
# Сохраним данные

In [19]:
#df.to_csv(wd + wd_data +'dataset_parsing_laptops_on_yand_market.csv')

In [ ]:
#Загрузка данных

In [21]:
df = pd.read_csv(wd + wd_data +'dataset_parsing_laptops_on_yand_market.csv').drop("Unnamed: 0", axis = 1)

In [23]:
# Посмотрим датасет
df

,title,about,price,rating,reviews,market,link,req
0,"Ноутбук HONOR MagicBook X 16 2024 16"" DOS Inte...",{},35630.0,4.8,226.0,yand,https://market.yandex.ru/product--noutbuk-hono...,Ноутбук
1,"Ноутбук для работы и учебы 15.6"" Echips Tagana...",{},24347.0,4.7,949.0,yand,https://market.yandex.ru/product--noutbuk-dlia...,Ноутбук
2,"Ноутбук для работы и учебы 14"" Echips Taganay,...",{},22310.0,4.7,949.0,yand,https://market.yandex.ru/product--noutbuk-dlia...,Ноутбук
3,"Мощный ноутбук для работы, учебы и игр Echips ...",{},35696.0,4.8,124.0,yand,https://market.yandex.ru/product--moshchnyi-no...,Ноутбук
4,"15.6"" Ноутбук Vitumi LV5PIW 1920x1080, Intel C...",{},21780.0,4.6,1298.0,yand,https://market.yandex.ru/product--15-6-noutbuk...,Ноутбук
...,...,...,...,...,...,...,...,...
314,Ноутбук Acer Aspire 15 A15-41M-R78V Ryzen 3 73...,"{'Экран:': '15.6"" (1920x1080) IPS, 60', 'Проце...",55529.0,4.6,11.0,yand,https://market.yandex.ru/product--noutbuk-acer...,laptop
315,Ноутбук Microsoft Surface Laptop 6 13.5 Intel ...,"{'Экран:': '13.5"" (2256x1504)', 'Процессор:': ...",183258.0,5.0,1.0,yand,https://market.yandex.ru/product--noutbuk-micr...,laptop
316,"Ноутбук игровой DIVAO 15,6 Intel Core i5 1240P...","{'Экран:': '15.6"" (1920x1080) IPS, 60 Гц', 'Пр...",58702.0,4.5,45.0,yand,https://market.yandex.ru/product--noutbuk-igro...,laptop
317,Asus Zenbook 14X i9-13900HIris Xe 32Gb 1Tb 2.8...,"{'Экран:': '14.5"" (2880x1800) OLED, 120 Гц', '...",127399.0,4.8,85.0,yand,https://market.yandex.ru/product--asus-zenbook...,laptop


In [25]:
# Достанем ссылку и проверим соответствие данных товара
print(df.loc[53])
df.loc[53]["link"]

title      Ноутбук для работы и учебы Echips Taganay 15.6...
about      {'Экран:': '15.6" (1920x1080) IPS, 60 Гц', 'Пр...
price                                                21340.0
rating                                                   4.7
reviews                                                949.0
market                                                  yand
link       https://market.yandex.ru/product--noutbuk-dlia...
req                                                  Ноутбук
Name: 53, dtype: object


'https://market.yandex.ru/product--noutbuk-dlia-raboty-i-ucheby-echips-taganay-15-6-1920x1080-ips-intel-celeron-n5095-8gb-ram-ssd-256gb-win-11-home/802134617?hid=90401&sku=103575066573&show-uid=17467151380470429996406020&from=search&cpa=1&do-waremd5=XToNkN2KIcxavGb_cfkjHw&sponsored=1&cpc=5vxclPOE20oK-tYfZPggWY1WtcjR9K-KJ9ebEqzOIlVIE4G5XzeFVmE9n5TSPN66H0ueoSDym-EXxMtNBlNBNb0YwgJi9C53b7Pd36Tz-fCZqxshb2_KUJBRNljkBwRpw_MynNZZIFTXOPPSNqwcyNFMVa7uJKgz0iN-W4Zp4njbPWTPQZezhg9cBtp8Geh6vueHyMOOANFVP0F5J1mHGsINnshlJQj2QCNBepe9X4kUj7xjLWGC2s59mDf_UAKw2juWzj-u8S81ac7qNkb3x2iIRMOvsLRhFE5O_0ucG7la2I3IXCjvmU2_yyMq-py_rG7kXbvNy0ipMeJuNAa4ltf6z0g3pKpq9acjwoFpo-CEl4pQ9OP91TRzV9TL7Ng7-MqXTfQ6BOnYD2vqRsEflK3kgfpxq72z7StDM2O4DZZj777-pSN38qhSUf_a8UgNa4_zo8ZJw8t4CzrNBEZJLFdnjEPU0lx88QIeOyGiB8wTMzhIf9WoIuYXW_YmjwlnAHPmfRK-QkPFaoG43dV9wqO7ZL-69OSy4yaWHViKXLg%2C&cc=CiAxMjZhMWExNzk3MWJlODYyZjU4MzIyODIxNDg5NzQ0ZBDnAYB95u0G&uniqueId=89672278'

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   title    319 non-null    object 
 1   about    319 non-null    object 
 2   price    319 non-null    float64
 3   rating   319 non-null    float64
 4   reviews  319 non-null    float64
 5   market   319 non-null    object 
 6   link     319 non-null    object 
 7   req      319 non-null    object 
dtypes: float64(3), object(5)
memory usage: 20.1+ KB


In [31]:
# Поменяем тип данных на корректныe
df.price = df.price.astype(int)
df.reviews = df.reviews.astype(int)

In [33]:
print("\nПропуски в данных:")
print(df.isnull().sum())


Пропуски в данных:
title      0
about      0
price      0
rating     0
reviews    0
market     0
link       0
req        0
dtype: int64


In [39]:
#пропусков нет из-за того что при пропусках в парсинге вызывалось исключение и такие данные не шли в датасет

In [41]:
print("\nОписательные статистики:")
print(df.describe(include='all'))


Описательные статистики:
                                                    title about  \
count                                                 319   319   
unique                                                191   169   
top     Ноутбук для работы и учебы Echips Hot 15.6" 19...    {}   
freq                                                   11   131   
mean                                                  NaN   NaN   
std                                                   NaN   NaN   
min                                                   NaN   NaN   
25%                                                   NaN   NaN   
50%                                                   NaN   NaN   
75%                                                   NaN   NaN   
max                                                   NaN   NaN   

                price      rating      reviews market  \
count      319.000000  319.000000   319.000000    319   
unique            NaN         NaN          NaN      1  

In [43]:
#рейтинг в пятибальной шкале, максимум - 5.0, а минимум у нас 3.0 значит всё верно
#Количесиво отзывов нормальное, минимум 1, так как товары без отзывов не добавлялись в датафрейм при парсинге
#Цена в 6 тысяч настораживает, но имеет место быть

In [45]:
#Посмотрим самый дешёвый и дорогой ноутбуки

In [47]:
df[df.price== 6434]["link"].values

array(['https://market.yandex.ru/product--10-1-netbuk-samsung-n102sp-1024x600-intel-atom-n2100-1x1-6ggts-ram-2-gb-hdd-320-gb-chernyi-windows-7/844800881?hid=90401&sku=103621661766&show-uid=17467151465222516861106009&from=search&cpa=1&do-waremd5=oLdUv6H1sWV4jNXiooo9qg&cpc=b2R4xrO6ApC2nj8XRSW4AOdbrKk-1UPm1DrSfCtVDnQoeYJ81HfW1rt23QOUVOURmYsXQtHkVwfAMpPXbU0lGbK8QReZsksrBD2o04Jbanvio5yOmH0TLIIgZaNO8mlaLHghxeYtLahODMzcpodMjCYHDZO7o-M2jXk_n4yTG85p9NlaBkg_y2HmfT6-826hW0yM3LPqdChju03esAMnVXpOZbqUthOFhuPY1qYji7cudC-o7M2bJSYBNlvjF5OepPqcVr4wbYJX8oYxsFCZmfMcHoOhUBUxyrnYuygvph2Gvi-HDVwMAAD3Jl4cpCDr15lTNV6N6c3rJaNe079LOGmep9h3M9Hts8qz09kDyIib1BEWLiAuYxFVNy4s_L9KYxFAA1NPGJ2RbJXhNkACLxf1SLDfYS7PrEcB44W0NyL6RiqepCgYPvHq8mRNo_QMCIkcKI0reYDPgucirla-FLVWgWb1YpLF1TOYq-ac7bwDJMR2HGJ7To8F3Gx8YLMr0E2S8HPnFZA1c6xN9ax22NXFXwkILdVO&cc=CiBhNGFiMjZiMWY5NmM2MjA1MTY5YTdmZjQwOThjNDRiZBAHgH3m7QY%2C&uniqueId=82452083&resale_goods=resale_resale&resale_goods_condition=resale_well'],
      dtype=object)

In [49]:
df[df.price== 322245]["link"].values

array(['https://market.yandex.ru/product--noutbuk-microsoft-surface-laptop-studio-2-i7-13700h-64gb-2tb-ssd-nvidia-geforce-rtx-4060-8gb-platinum/15891780?hid=90401&sku=103713212014&show-uid=17467151641637566156806021&from=search&cpa=1&do-waremd5=c-8AxEsr7npbrDxzLtxeXw&sponsored=1&cpc=hy31mIFHFxFiJ8pdofedAf1LB0kFlWSpjWZrGFCARflMSja_mDMpbzFkJl4XX4xL4vSRaTtvuV1PZZyvSunidfgHcqg_1-nyme7qsBENBIvIX8xMC9diSsN4nY_l2z5k4LpzShTJJRZkrzAxwYV7k2TVvHrX3dwFHGGxARoP4HeSGjiFEvYrHMPD_u_MH7UkGXhIn8QSkdoyLD5klH1sxMQ6Y52PEaDLnztY75KaCuvdefwqToKYCQ0ufjcw7ofPfNDkoHKfvdUXKgeW4X8JuHVUenMRRdHhfk_yzIvugtvCW811Q6bTWaNEKhzd4SA__6CI0HyLpehB6WvHZzJulVIOYnY56CTZ5Dg7T847O0ON7Is-EXFYwtnrLFfscNabi2xLjfPEJ-ZZzbkY3YhxxqEN1p6tprsXgqvLNcxnCH3Dn2-DMlhd2u-AH3qP5VB5jsUVBAyx1f34oYSRPvKCRpVH4UWTtA4bGw7Mi6rZYysz0_MoL94TdDzkLRMMSo5025aFitxdBMwAEq93Mz-tlDmmKWP_N4XpqwkZWGrQG9c%2C&cc=CiA0M2FjODA5ZTlhNDc4OWY5OTU1N2M5NzU1MWQ1ZDkxOBDnAYB95u0G&uniqueId=182394256'],
      dtype=object)

In [51]:
# Удалим данные с пропусками(на всякий случай) и дубликаты
df = df.dropna()
df = df.drop_duplicates()

def Is_there_any_truth(lst):
    for i in lst:
        if i == True:
            return True
    return False


#Очистим от товаров не содержащих интересующее нас слова
list_request = ['Ноутбук', 'Нетбук', 'laptop']
df = df[df.apply(lambda x: Is_there_any_truth([i.lower() in x['title'].lower() for i in list_request]), axis=1)]

#Очистим данные от выбросов

def clearing(df, reqvest = None, col_name = "price"):
    if reqvest != None:
        filtred_df = df[df.req == reqvest]
    else:
        filtred_df = df
    
    q1 = filtred_df[col_name].quantile(0.25)
    q3 = filtred_df[col_name].quantile(0.75)
    iqr = q3 - q1
    df = df[((df[col_name] < q3 + 1.5 * iqr) & (df[col_name] > q1 - 1.5 * iqr)) | (df.req != reqvest)]
    return df

df = clearing(df)
df

,title,about,price,rating,reviews,market,link,req
0,"Ноутбук HONOR MagicBook X 16 2024 16"" DOS Inte...",{},35630,4.8,226,yand,https://market.yandex.ru/product--noutbuk-hono...,Ноутбук
1,"Ноутбук для работы и учебы 15.6"" Echips Tagana...",{},24347,4.7,949,yand,https://market.yandex.ru/product--noutbuk-dlia...,Ноутбук
2,"Ноутбук для работы и учебы 14"" Echips Taganay,...",{},22310,4.7,949,yand,https://market.yandex.ru/product--noutbuk-dlia...,Ноутбук
3,"Мощный ноутбук для работы, учебы и игр Echips ...",{},35696,4.8,124,yand,https://market.yandex.ru/product--moshchnyi-no...,Ноутбук
4,"15.6"" Ноутбук Vitumi LV5PIW 1920x1080, Intel C...",{},21780,4.6,1298,yand,https://market.yandex.ru/product--15-6-noutbuk...,Ноутбук
...,...,...,...,...,...,...,...,...
313,Ноутбук Microsoft Surface Laptop 7 Copilot + P...,"{'Экран:': '15"" (2496x1664) IPS, 120 Гц', 'Про...",169442,5.0,1,yand,https://market.yandex.ru/product--noutbuk-micr...,laptop
314,Ноутбук Acer Aspire 15 A15-41M-R78V Ryzen 3 73...,"{'Экран:': '15.6"" (1920x1080) IPS, 60', 'Проце...",55529,4.6,11,yand,https://market.yandex.ru/product--noutbuk-acer...,laptop
315,Ноутбук Microsoft Surface Laptop 6 13.5 Intel ...,"{'Экран:': '13.5"" (2256x1504)', 'Процессор:': ...",183258,5.0,1,yand,https://market.yandex.ru/product--noutbuk-micr...,laptop
316,"Ноутбук игровой DIVAO 15,6 Intel Core i5 1240P...","{'Экран:': '15.6"" (1920x1080) IPS, 60 Гц', 'Пр...",58702,4.5,45,yand,https://market.yandex.ru/product--noutbuk-igro...,laptop


In [53]:
# Анализ числовых переменных у очищенных данных
numeric_cols = ['price', 'rating', 'reviews']
for col in numeric_cols:
    print(f"\nАнализ столбца {col}:")
    print(f"Медиана: {df[col].median()}")
    print(f"Среднее: {df[col].mean()}")
    print(f"Стандартное отклонение: {df[col].std()}")
    print(f"Минимум: {df[col].min()}")
    print(f"Максимум: {df[col].max()}")
    print(f"Количество нулевых значений: {(df[col] == 0).sum()}")


Анализ столбца price:
Медиана: 35696.0
Среднее: 50462.33650793651
Стандартное отклонение: 36145.82931127432
Минимум: 6434
Максимум: 322245
Количество нулевых значений: 0

Анализ столбца rating:
Медиана: 4.8
Среднее: 4.713015873015872
Стандартное отклонение: 0.325833741555076
Минимум: 3.0
Максимум: 5.0
Количество нулевых значений: 0

Анализ столбца reviews:
Медиана: 96.0
Среднее: 304.34603174603177
Стандартное отклонение: 408.0110889063049
Минимум: 1
Максимум: 1510
Количество нулевых значений: 0


In [55]:
# Значения не вызывают подозрений - всё в пределах нормы

In [57]:
df

,title,about,price,rating,reviews,market,link,req
0,"Ноутбук HONOR MagicBook X 16 2024 16"" DOS Inte...",{},35630,4.8,226,yand,https://market.yandex.ru/product--noutbuk-hono...,Ноутбук
1,"Ноутбук для работы и учебы 15.6"" Echips Tagana...",{},24347,4.7,949,yand,https://market.yandex.ru/product--noutbuk-dlia...,Ноутбук
2,"Ноутбук для работы и учебы 14"" Echips Taganay,...",{},22310,4.7,949,yand,https://market.yandex.ru/product--noutbuk-dlia...,Ноутбук
3,"Мощный ноутбук для работы, учебы и игр Echips ...",{},35696,4.8,124,yand,https://market.yandex.ru/product--moshchnyi-no...,Ноутбук
4,"15.6"" Ноутбук Vitumi LV5PIW 1920x1080, Intel C...",{},21780,4.6,1298,yand,https://market.yandex.ru/product--15-6-noutbuk...,Ноутбук
...,...,...,...,...,...,...,...,...
313,Ноутбук Microsoft Surface Laptop 7 Copilot + P...,"{'Экран:': '15"" (2496x1664) IPS, 120 Гц', 'Про...",169442,5.0,1,yand,https://market.yandex.ru/product--noutbuk-micr...,laptop
314,Ноутбук Acer Aspire 15 A15-41M-R78V Ryzen 3 73...,"{'Экран:': '15.6"" (1920x1080) IPS, 60', 'Проце...",55529,4.6,11,yand,https://market.yandex.ru/product--noutbuk-acer...,laptop
315,Ноутбук Microsoft Surface Laptop 6 13.5 Intel ...,"{'Экран:': '13.5"" (2256x1504)', 'Процессор:': ...",183258,5.0,1,yand,https://market.yandex.ru/product--noutbuk-micr...,laptop
316,"Ноутбук игровой DIVAO 15,6 Intel Core i5 1240P...","{'Экран:': '15.6"" (1920x1080) IPS, 60 Гц', 'Пр...",58702,4.5,45,yand,https://market.yandex.ru/product--noutbuk-igro...,laptop


In [63]:
#сохраним очищенные данные
#df.to_csv(wd + wd_data +'clean_dataset_parsing_laptops_on_yand_market.csv')

In [65]:
# Загрузим очищенные данные
df = pd.read_csv(wd + wd_data +'clean_dataset_parsing_laptops_on_yand_market.csv').drop("Unnamed: 0", axis = 1)